# HASH CODE BOOKS AND LIBRARIES 

l'obbiettivo è ottimizzare la scansione dei libri.

Ogni libro verra assegnato ad un libreria e avrà i seguenti attributi:
B= numeri di libri differenti
ID = da 0 a B-1

### Vincolo 1 :
scansionare ogni libro una sola volta

 L= numero di librerie
 SB = set libri in ogni libreria
 T = tempo per iscrivere alla libreria
 N = numero di libri scansionati/day

### Vincolo 2 :
solo una libreria alla volta puo essere iscritta

## SOLUTION

1. LETTURA INPUT
2. Calcola un punteggio euristico per ogni biblioteca con questa formula:

   
   $$ Score[biblioteca] = (SumLibri[biblioteca])/(Registrazione[biblioteca]) $$


   dove library_books_score è calcolato in questo modo:

   1. Calcola i giorni disponibili per questa biblioteca:

   $$ GiorniDisponibili[biblioteca] = (Giorno) - (Registrazione[biblioteca])  $$


   2. Conta quanti libri possono essere scansionati in questi giorni:

  
   $$ K[biblioteca] = (GiorniDisponibili[biblioteca])*(Maxlibriscan[biblioteca])  $$


   3. Ottieni i primi k libri in base al loro punteggio e somma i loro punteggi.

3. Ordina le biblioteche in base al loro punteggio
4. Per ogni biblioteca in ordine:
   seleziono i libri con punteggio piu alto che possono essere scansionati entro la deadline

5. CREA l'output


In [1]:
import numpy as np
import operator
import sys

def ordinamento(array_1, array_2):
    for i in range(len(array_1)):
        for j in range(len(array_2)):
            #print(array_1)
            if array_1[i]>array_1[j]:
                #print("scambio")
                array_1[i], array_1[j]= array_1[j], array_1[i]
                array_2[i], array_2[j]= array_2[j], array_2[i]
    return array_1 , array_2



##### CLASSE LIBRARY

###### ATTRIBUTI :
    ID : id libreria
    id_libri : insieme id dei libri
    registrazione : giorno della registrazione
    MAX_BOOK_SCANNED_PER_DAY : quantita massima di libri/giorno

###### VARIABILI GLOBALI :
    punteggio_libri :
    biblioteche_max_books_scanned_per_day: libri max scannerizzati 
    D : Deadline
    libri_totali: numero di libri esistenti
    giorno : giorno attuale

###### METODI :
    top_punteggio_libri : somma del punteggio in base al giorno di fine registrazine
    top_books_id: libri con punteggio piu elevato
    somma_punteggio_libri :somma dei punteggi piu elevati

In [2]:
class Biblioteca:

#iniziallizzo
    def __init__(self, id, id_libri, registrazione, scanned_su_day):
        self.id = id
        self.id_libri = id_libri
        self.registrazione = registrazione
        self.scanned_su_day = scanned_su_day

#definisco il miglior libro
    def top_libri_id(self, giorno=0):
        global punteggio_libri, registrazione, biblioteche_max_books_scanned_su_day, D, libri_totali
        
        giorni_disponibili = D - self.registrazione - giorno
        
        #elimino i libri da quelli disponibili
        #Set è una raccolta non ordinata, non modificabile* e non indicizzata. Nessun membro duplicato.
        libri_disponibili = list(set(self.id_libri) - set(libri_totali))
        #prende lo score solamente dei libri disponibili
        punteggio_attuale = np.take(punteggio_libri, libri_disponibili)

        #giorni disponibili*num_max_book/day per ogni oggetto e prendo il max
        max_num_libri_scan = max(min(int(giorni_disponibili * self.scanned_su_day), len(libri_disponibili)), 0)

        if max_num_libri_scan == 0:
            return []

        # get top k books (k=max_num_libri_scan)
        ind = np.argpartition(punteggio_attuale, -max_num_libri_scan)[-max_num_libri_scan:]

        #ritorna i libri disponibili per ID
        return np.take(libri_disponibili, ind)

#definisco il miglior score
    def top_punteggio_libri(self, giorno=0):

          # DEF VARIABILI
        global punteggio_libri, registrazione, biblioteche_max_books_scanned_su_day, D
        giorni_disponibili = D - self.registrazione - giorno

        # NUMERO MASSIMO DI LIBRI SCANSIONABILI
        max_num_libri_scan = max(min(int(giorni_disponibili * self.scanned_su_day), len(self.id_libri)), 0)

        #ritorna un array contenente gli score solamente dei libri presenti nella libreria
        punteggio_attuale = np.take(punteggio_libri, self.id_libri)

        # get top k books (k=max_num_libri_scan)
        #ritorna gli id da 0 a max num books dei libri con lo score maggiore
        ind = np.argpartition(punteggio_attuale, -max_num_libri_scan)[-max_num_libri_scan:]
        
        best_books_scores = np.take(punteggio_attuale, ind)

        return np.sum(best_books_scores)

    def __repr__(self):
        return self.id.__str__()

    def punteggio_iterativo(self, start_day=0):
        global punteggio_libri, registrazione, biblioteche_max_books_scanned_su_day, D, libri_totali
        
        giorni_disponibili = D - self.registrazione - giorno
        libri_disponibili = list(set(self.id_libri) - set(libri_totali))
        # NUMERO MASSIMO DI LIBRI SCANSIONABILI
        max_num_libri_scan = max(min(int(giorni_disponibili * self.scanned_su_day), len(self.id_libri)), 0)

        punteggio_attuale = np.take(punteggio_libri, self.id_libri)

        # get top k books (k=max_num_books)
        ind = np.argpartition(punteggio_attuale, -max_num_libri_scan)[-max_num_libri_scan:]
        best_books_scores = np.take(punteggio_attuale, ind)
        totale=np.sum(best_books_scores)
        return totale

In [3]:
#sommo gli score dei libri
def somma_punteggio_libri(id_libri):
    global punteggio_libri
    return np.sum(np.take(punteggio_libri, list(id_libri)))

files = ["a_example", "b_read_on", "c_incunabula", "d_tough_choices", "e_so_many_books", "f_libraries_of_the_world"]
files = ["b_read_on"]
totale_punteggio = 0

In [4]:
for file in files:

## LETTURA DA FILE
    with open("inputs/" + file + ".txt", "r") as f:
        content = f.read().splitlines()
    print(file)

    libri_esistenti, tot_biblio, D = list(map(int, content[0].split(' ')))

    punteggio_libri = list(map(int, content[1].split(' ')))
    pos = 1
    numero_libri = np.zeros(tot_biblio)
    registrazione = np.zeros(tot_biblio)
    biblioteche_scanned_su_day = np.zeros(tot_biblio)
    biblioteche = np.empty(tot_biblio, dtype=Biblioteca)
    for i in range(tot_biblio):
        pos += 1
        n, t, m = list(map(int, content[pos].split(' ')))
        numero_libri[i] = n
        registrazione[i] = t
        biblioteche_scanned_su_day[i] = m
        
        pos += 1
        id_libri = np.asarray(list(map(int, content[pos].split(' '))))
        biblioteche[i] = Biblioteca(i, id_libri, t, m)

## INIZIO PROGRAMMA
    sys.stdout.write("\rOrdinamento...")
    
    biblioteca_punteggio_libri = np.vectorize(lambda biblioteca: biblioteca.top_punteggio_libri())
    biblioteche_scores = biblioteca_punteggio_libri(biblioteche)

    Euristica = np.vectorize(lambda book_score, signup_days: book_score / signup_days)
    punteggio_totale = Euristica(biblioteche_scores, registrazione)

    punteggio_totale, biblioteche_ordinate = ordinamento(punteggio_totale, biblioteche)


    a=5 # distanza gruppi ordinamento
    b=4 # elementi ordinati
### SALVATAGGIO FILE
    cnt=10 # spiazzamento inizio
    sx=dx=1000
    temp = 0
    libri_totali = set()
    with open("outputs/" + file + ".out", 'w+') as f:
        f.write(str(tot_biblio) + "\n")
        giorno = 0
        for i in range(tot_biblio) :
            if giorno<=D:

                if i==cnt:
                    cnt+=a
                    sx=cnt-a+1
                    dx=cnt-a+b+1
                    Euristica = np.vectorize(lambda biblioteca: biblioteca.punteggio_iterativo())
                    punteggio_totale = Euristica(biblioteche_ordinate[i+1:i+b+1])
                    punteggio_totale_2, biblioteche_ordinate_2 = ordinamento(punteggio_totale, biblioteche_ordinate[i+1:i+b+1])

                if i>=sx and i<dx :
                    biblioteca_attuale =  biblioteche_ordinate_2[i-sx]
                else :
                    biblioteca_attuale = biblioteche_ordinate[i]
                
                chosen_id_libri = biblioteca_attuale.top_libri_id(giorno)
                libri_totali.update(chosen_id_libri)
                giorno += biblioteca_attuale.registrazione 
                    

                if len(chosen_id_libri) > 0:
                    f.write(str(biblioteca_attuale.id) + " " + str(len(chosen_id_libri)) + "\n")
                    f.write(str(' '.join(map(str, chosen_id_libri))) + "\n")
                else:
                    f.write(str(biblioteca_attuale.id) + " 1\n")
                    f.write(str(biblioteca_attuale.id_libri[0])+"\n")
                c=i+1
                progress = 100 * (i+1) / (tot_biblio)
                sys.stdout.write("\r Caricamento                  (" + str(int(progress)) + " %)")

        punteggio = somma_punteggio_libri(libri_totali)
        totale_punteggio += punteggio

        print("\r-  Punteggio:        ", punteggio)
        print("                totale biblioteche iscritte:        ",  c  , " su  " , tot_biblio)

print("")
print("totale punteggio:", totale_punteggio)


b_read_on
-  Punteggio:         5822900 (91 %)
                totale biblioteche iscritte:         91  su   100

totale punteggio: 5822900
